In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon May 15 23:18:41 2017

@author: zhouyi
"""
import glob
import os
# if you are using OPENBLAS, you might want to turn this option on. Otherwise, joblib might get stuck
# os.environ['OPENBLAS_NUM_THREADS'] = '1'

import numpy as np
import scipy.sparse
import pandas as pd

import expomf


In [2]:
DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/ml-1m'

unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
    
unique_sid = list()
with open(os.path.join(DATA_ROOT, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_songs = len(unique_sid)
n_users = len(unique_uid)

In [3]:
def load_data(csv_file, shape=(n_users, n_songs)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['uid'], dtype=np.int32), np.array(tp['sid'], dtype=np.int32)
    count = tp['rating']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.csv'))
# binarize the data
train_data.data = np.ones_like(train_data.data)

print train_data.shape
print train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.csv'))
# binarize the data
vad_data.data = np.ones_like(vad_data.data)

test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.csv'))
# binarize the data
test_data.data = np.ones_like(test_data.data)

(4200, 2019)
(636096,)


In [4]:
n_components = 18
max_iter = 15
n_jobs = 5
# grid search on init_mu = {0.1, 0.05, 0.01, 0.005, 0.001} from validation set
init_mu = 0.005
lam = 1e-5

save_dir="ExpoMF_ML_K%d_lam%1.0E_initmu%1.0E" % (n_components, lam, init_mu)

coder = expomf.ExpoMF(n_components=n_components, max_iter=max_iter, batch_size=10, init_std=0.01, 
                      n_jobs=n_jobs, random_state=98765, save_params=True, 
                      save_dir=save_dir, early_stopping=True, verbose=True, 
                      lam_y=1., lam_theta=lam, lam_beta=lam, a=1., b=1., init_mu=init_mu)

coder.fit(train_data, vad_data=vad_data, k=20)

ITERATION #0
	Updating user factors: time=2.20
	Updating item factors: time=1.52
	Updating exposure prior: time=0.38
	Validation NDCG@k: 0.0101
ITERATION #1
	Updating user factors: time=1.41
	Updating item factors: time=1.10
	Updating exposure prior: time=0.26
	Validation NDCG@k: 0.0410
ITERATION #2
	Updating user factors: time=1.32
	Updating item factors: time=1.21
	Updating exposure prior: time=0.38
	Validation NDCG@k: 0.0493
ITERATION #3
	Updating user factors: time=1.63
	Updating item factors: time=1.42
	Updating exposure prior: time=0.32
	Validation NDCG@k: 0.0515
ITERATION #4
	Updating user factors: time=1.31
	Updating item factors: time=1.11
	Updating exposure prior: time=0.26
	Validation NDCG@k: 0.0530
ITERATION #5
	Updating user factors: time=1.31
	Updating item factors: time=1.11
	Updating exposure prior: time=0.26
	Validation NDCG@k: 0.0537
ITERATION #6
	Updating user factors: time=1.32
	Updating item factors: time=1.00
	Updating exposure prior: time=0.27
	Validation NDCG@k:

ExpoMF(batch_size=10, early_stopping=True, init_std=0.01, max_iter=15,
    n_components=18, n_jobs=5, random_state=98765,
    save_dir='ExpoMF_ML_K18_lam1E-05_initmu5E-03', save_params=True,
    verbose=True)

In [5]:
'''More Results are summarized here:

Test NDCG@k: 0.13427
Test Recall@k: 0.14925
Test MAP@k: 0.13630


'''

'More Results are summarized here:\n\nTest NDCG@k: 0.13427\nTest Recall@k: 0.14925\nTest MAP@k: 0.13630\n\n\n'